In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b1/1/tf_efficientnet_b1_aa-ea7a6ee0.pth
/kaggle/input/health-data-chatbot/mentalhealth.csv
/kaggle/input/health-data-chatbot/mentalhealth.json


In [2]:
import re
import nltk
from nltk import pos_tag,word_tokenize
from nltk.stem import wordnet
from nltk.corpus import stopwords
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [3]:
df = pd.read_csv('/kaggle/input/health-data-chatbot/mentalhealth.csv')
df

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,"Mental illness does can affect anyone, regardl..."
2,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
3,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."
4,1619387,What should I do if I know someone who appears...,We encourage those with symptoms to talk to th...
...,...,...,...
92,4373204,How do I know if I'm drinking too much?,Sorting out if you are drinking too much can b...
93,7807643,"If cannabis is dangerous, why are we legalizin...","Cannabis smoke, for example, contains cancer-c..."
94,4352464,How can I convince my kids not to use drugs?,You can't. But you can influence their capacit...
95,6521784,What is the legal status (and evidence) of CBD...,Cannabidiol or CBD is a naturally occurring co...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Question_ID  97 non-null     int64 
 1   Questions    97 non-null     object
 2   Answers      97 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [5]:
df.describe()

,Question_ID
count,9.700000e+01
mean,5.115247e+06
std,2.707777e+06
min,1.030153e+06
25%,2.884887e+06
50%,4.759773e+06
75%,7.657263e+06
max,9.679704e+06


In [6]:
df.isnull().sum()

Question_ID    0
Questions      0
Answers        0
dtype: int64

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
s = 'tell me about yourself'
words= word_tokenize(s)
print(words)

['tell', 'me', 'about', 'yourself']


In [9]:
nltk.download('wordnet')
import zipfile

with zipfile.ZipFile('/usr/share/nltk_data/corpora/wordnet.zip', 'r') as zip_ref:
    zip_ref.extractall('/usr/share/nltk_data/corpora')
    print("done")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
done


In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [12]:
lema_words = []
def text_normalization(text):
    text = str(text).lower()                        # text to lower case
    spl_char_text = re.sub(r'[^ a-z]','',text)      # removing special characters
    tokens = nltk.word_tokenize(spl_char_text)      # word tokenizing
    lema = wordnet.WordNetLemmatizer()              # intializing lemmatization
    tags_list = pos_tag(tokens,tagset=None)         # parts of speech
    lema_words = []                                 # empty list 
    for token,pos_token in tags_list:               # lemmatize according to POS
        if pos_token.startswith('V'):               # Verb
            pos_val = 'v'
        elif pos_token.startswith('J'):             # Adjective
            pos_val = 'a'
        elif pos_token.startswith('R'):             # Adverb
            pos_val = 'r'
        else:
            pos_val = 'n'                           # Noun
        lema_token = lema.lemmatize(token,pos_val)

        if lema_token in stop: 
          lema_words.append(lema_token)             # appending the lemmatized token into a list
    
    return " ".join(lema_words) 

In [13]:
text_normalization('telling you some stuffs about me')  # example

'you some about me'

In [14]:
df['lemmantized_text'] = df['Questions'].apply(text_normalization)
df.head()


,Question_ID,Questions,Answers,lemmantized_text
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...,what do it to have a
1,2110618,Who does mental illness affect?,"Mental illness does can affect anyone, regardl...",who do
2,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...,what be some of the of
3,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi...",can with
4,1619387,What should I do if I know someone who appears...,We encourage those with symptoms to talk to th...,what should i do if i who to have the of a


In [15]:
cv= CountVectorizer()
X = cv.fit_transform(df['lemmantized_text']).toarray()

In [16]:
features = cv.get_feature_names_out()
df_bow = pd.DataFrame(X,columns = features)
df_bow

,about,after,an,and,any,be,before,between,but,can,...,to,too,we,what,when,where,who,why,with,you
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
93,0,0,0,0,0,2,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
94,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
95,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [17]:
Ouestion = 'What treatment options are available'

In [18]:
Question_lemma = text_normalization(Ouestion)
Question_bow = cv.transform([Question_lemma]).toarray()

In [19]:
cosine_values = 1-pairwise_distances(df_bow,Question_bow,metric = 'cosine')
(cosine_values)

array([[0.31622777],
       [0.        ],
       [0.5       ],
       [0.        ],
       [0.23570226],
       [0.        ],
       [1.        ],
       [0.31622777],
       [0.70710678],
       [0.        ],
       [0.31622777],
       [0.        ],
       [0.40824829],
       [0.25      ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.70710678],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [0.26726124],
       [0.        ],
       [0.40824829],
       [0.70710678],
       [0.25      ],
       [0.        ],
       [0.28867513],
       [0.23570226],
       [0.        ],
       [0.31622777],
       [0.        ],
       [0.35355339],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [20]:
index_values = cosine_values.argmax()
index_values

6

In [21]:
df['Answers'].iloc[index_values]

'Different treatment options are available for individuals with mental illness.'

In [22]:
## TFID
question = 'What treatment options are available'

In [23]:
# using tf-idf
tfidf = TfidfVectorizer()
x_tfidf = tfidf.fit_transform(df['lemmantized_text']).toarray()

In [24]:
Question_lemma1 = text_normalization(question)
Question_tfidf = tfidf.transform([Question_lemma1]).toarray()

In [25]:
df_tfidf = pd.DataFrame(x_tfidf,columns = tfidf.get_feature_names_out())
df_tfidf.head()

,about,after,an,and,any,be,before,between,but,can,...,to,too,we,what,when,where,who,why,with,you
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.451394,0.0,0.0,0.302478,0.0,0.0,0.000000,0.0,0.00000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.877701,0.0,0.00000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.243612,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.213472,0.0,0.0,0.000000,0.0,0.00000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.407601,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.91316,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.309284,0.0,0.0,0.207251,0.0,0.0,0.471012,0.0,0.00000,0.0


In [26]:
cos = 1-pairwise_distances(df_tfidf,Question_tfidf,metric='cosine')

array([[0.19934817],
       [0.        ],
       [0.32390963],
       [0.        ],
       [0.13658848],
       [0.        ],
       [1.        ],
       [0.19419492],
       [0.60151332],
       [0.        ],
       [0.19419492],
       [0.        ],
       [0.20023572],
       [0.17842037],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.57482171],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [0.19775935],
       [0.        ],
       [0.29108153],
       [0.65904938],
       [0.17156074],
       [0.        ],
       [0.1796983 ],
       [0.14501087],
       [0.        ],
       [0.30481321],
       [0.        ],
       [0.28362797],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [27]:
index_value_tfidf = cos.argmax()
index_value_tfidf

6

In [28]:
df['Answers'].iloc[index_value_tfidf]

'Different treatment options are available for individuals with mental illness.'

In [29]:
def chat_bow(text):
    lemma = text_normalization(text)
    bow = cv.transform([lemma]).toarray()
    cosine_similarity=1-pairwise_distances(df_bow,bow,metric ='cosine')
    index_value = cosine_similarity.argmax()
    return df['Answers'].loc[index_value]

In [30]:
chat_bow('can you prevent mental health problems')

'We can all suffer from mental health challenges, but developing our wellbeing, resilience, and seeking help early can help prevent challenges becoming serious.'

In [31]:
chat_bow('what is mental health')

'Different treatment options are available for individuals with mental illness.'

In [32]:
def chat_tfidf(text):
    lemma = text_normalization(text)
    tf = tfidf.transform([lemma]).toarray()
    cos =1- pairwise_distances(df_tfidf,tf,metric = 'cosine')
    index_value = cos.argmax()
    return df['Answers'].loc[index_value]

In [33]:
chat_tfidf('what is mental health')

'Different treatment options are available for individuals with mental illness.'

In [34]:
chat_tfidf('what is vaping')

'Different treatment options are available for individuals with mental illness.'

In [35]:
chat_tfidf('how do i see a counsellor')

'If your beliefs , thoughts , feelings or behaviours have a significant impact on your ability to function in what might be considered a normal or ordinary way, it would be important to seek help.'